In [1]:

import pandas as pd
import numpy as np
import torch
import os

In [2]:
import ultralytics

In [3]:
image_dir = 'cards_dataset/images/'

In [4]:
data_test = []
data_train = []
test_labels = pd.read_csv(image_dir+'test_labels.csv')
train_labels = pd.read_csv(image_dir+'train_labels.csv')

In [5]:
train_labels

,filename,width,height,class,xmin,ymin,xmax,ymax
0,cam_image1.jpg,480,270,queen,173,24,260,137
1,cam_image1.jpg,480,270,queen,165,135,253,251
2,cam_image1.jpg,480,270,ten,255,96,337,208
3,cam_image10.jpg,960,540,ten,501,116,700,353
4,cam_image10.jpg,960,540,queen,261,124,453,370
...,...,...,...,...,...,...,...,...
522,IMG_2700.JPG,378,504,ace,74,105,303,425
523,IMG_2701.JPG,378,504,jack,64,102,313,453
524,IMG_2702.JPG,378,504,nine,85,133,290,419
525,IMG_2703.JPG,378,504,ace,82,152,277,422


In [6]:
classes = train_labels['class'].unique()
classes

array(['queen', 'ten', 'nine', 'king', 'jack', 'ace'], dtype=object)

In [7]:
class_names = {'ace': 0,
 'king' : 1,
 'queen' : 2,
 'jack' :3 ,
 'ten': 4,
 'nine': 5}

Преобразуем в YOLO формат

In [8]:
def normalize_coordinates(df):
    df['x_center'] = ((df['xmin']+df['xmax'])/2) / df['width']
    df['y_center'] = ((df['ymin']+df['ymax'])/2) / df['height']
    df['bbox_width'] = (df['xmax']-df['xmin']) / df['width']
    df['bbox_height'] = (df['ymax']-df['ymin']) / df['height']
    return df


In [9]:
test_labels = normalize_coordinates(test_labels)
train_labels = normalize_coordinates(train_labels)

In [10]:
test_labels['class'] = test_labels['class'].map(class_names)
train_labels['class'] = train_labels['class'].map(class_names)

In [11]:
test_labels

,filename,width,height,class,xmin,ymin,xmax,ymax,x_center,y_center,bbox_width,bbox_height
0,cam_image2.jpg,960,540,1,312,30,485,249,0.415104,0.258333,0.180208,0.405556
1,cam_image2.jpg,960,540,1,514,24,694,245,0.629167,0.249074,0.187500,0.409259
2,cam_image2.jpg,960,540,1,305,263,489,519,0.413542,0.724074,0.191667,0.474074
3,cam_image2.jpg,960,540,1,515,267,704,523,0.634896,0.731481,0.196875,0.474074
4,cam_image4.jpg,960,540,3,297,18,459,237,0.393750,0.236111,0.168750,0.405556
...,...,...,...,...,...,...,...,...,...,...,...,...
138,IMG_2677.JPG,378,504,4,80,114,163,357,0.321429,0.467262,0.219577,0.482143
139,IMG_2678.JPG,378,504,3,119,228,275,445,0.521164,0.667659,0.412698,0.430556
140,IMG_2678.JPG,378,504,3,42,239,132,459,0.230159,0.692460,0.238095,0.436508
141,IMG_2686.JPG,378,504,2,143,150,319,378,0.611111,0.523810,0.465608,0.452381


In [12]:
train_labels

,filename,width,height,class,xmin,ymin,xmax,ymax,x_center,y_center,bbox_width,bbox_height
0,cam_image1.jpg,480,270,2,173,24,260,137,0.451042,0.298148,0.181250,0.418519
1,cam_image1.jpg,480,270,2,165,135,253,251,0.435417,0.714815,0.183333,0.429630
2,cam_image1.jpg,480,270,4,255,96,337,208,0.616667,0.562963,0.170833,0.414815
3,cam_image10.jpg,960,540,4,501,116,700,353,0.625521,0.434259,0.207292,0.438889
4,cam_image10.jpg,960,540,2,261,124,453,370,0.371875,0.457407,0.200000,0.455556
...,...,...,...,...,...,...,...,...,...,...,...,...
522,IMG_2700.JPG,378,504,0,74,105,303,425,0.498677,0.525794,0.605820,0.634921
523,IMG_2701.JPG,378,504,3,64,102,313,453,0.498677,0.550595,0.658730,0.696429
524,IMG_2702.JPG,378,504,5,85,133,290,419,0.496032,0.547619,0.542328,0.567460
525,IMG_2703.JPG,378,504,0,82,152,277,422,0.474868,0.569444,0.515873,0.535714


Формат: [class x_center y_center width height]

In [13]:
cols = [
 'class',
 'x_center',
 'y_center',
 'bbox_width',
 'bbox_height']

In [14]:
def pandas_to_dict(labels):
    res_dict = {}
    for key in labels['filename'].unique().tolist():
        res_dict[key] = []
    for i in range(len(labels)):
        filename = labels.iloc[i,:]['filename']
        res_dict[filename].append(labels.iloc[i,:][cols].to_list())
    return res_dict


In [15]:
labels_train_YOLO = pandas_to_dict(train_labels)
labels_test_YOLO = pandas_to_dict(test_labels)

In [16]:
labels_train_YOLO


{'cam_image1.jpg': [[2,
   0.4510416666666667,
   0.29814814814814816,
   0.18125,
   0.4185185185185185],
  [2,
   0.4354166666666667,
   0.7148148148148148,
   0.18333333333333332,
   0.42962962962962964],
  [4,
   0.6166666666666667,
   0.562962962962963,
   0.17083333333333334,
   0.4148148148148148]],
 'cam_image10.jpg': [[4,
   0.6255208333333333,
   0.43425925925925923,
   0.20729166666666668,
   0.4388888888888889],
  [2, 0.371875, 0.45740740740740743, 0.2, 0.45555555555555555]],
 'cam_image11.jpg': [[5,
   0.3723958333333333,
   0.45555555555555555,
   0.2760416666666667,
   0.5555555555555556]],
 'cam_image12.jpg': [[1,
   0.48020833333333335,
   0.4981481481481482,
   0.20625,
   0.4444444444444444]],
 'cam_image13.jpg': [[3,
   0.46822916666666664,
   0.49074074074074076,
   0.209375,
   0.45555555555555555]],
 'cam_image14.jpg': [[3,
   0.42135416666666664,
   0.5435185185185185,
   0.22395833333333334,
   0.4685185185185185]],
 'cam_image15.jpg': [[0,
   0.497916666666666

Для каждой картинки создадим текстовый файл с разметкой

In [17]:
def create_txt_labels(dict_labels ,dir = None):
    for key in dict_labels.keys():
        filename = key.split('.')[0]
        with open(dir + filename + '.txt', mode='w') as curr:
            for arr_of_labels in dict_labels[key]:
                for item in arr_of_labels:
                    curr.write(str(item)+' ')
                    
                curr.write('\n')

In [18]:
DIR_TRAIN = 'cards_dataset/labels/train/'
create_txt_labels(labels_train_YOLO, DIR_TRAIN)

In [19]:
DIR_TEST = 'cards_dataset/labels/val/'
create_txt_labels(labels_test_YOLO, DIR_TEST)